In [8]:
import numpy as np
import pandas as pd
import os

In [9]:
! nvidia-smi

Wed Jun 29 02:17:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 32%   41C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   

In [10]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Data

In [11]:
def get_forb(f_data,com_ls):
    forb_ls =[]

    for i in com_ls:
        if (f_data["Company"]==i).any():
            if (f_data[f_data.Company==i]["Industry"]=="Semiconductors").any():
                forb_ls.append(1)
            elif (f_data[f_data.Company==i]["Sector"]=="Information Technology").any():
                forb_ls.append(2)
            else:
                forb_ls.append(3)
        else:
            forb_ls.append(4)
    
    return forb_ls

In [12]:
correct_ls = []
correct_ls2 = []

for i in range(2015,2021):
    now = "competitor_df/lda_cn_{year}.csv".format(year=i)
    lda_now = pd.read_csv(now)
    f = "../patent/code/依年分/Forbes/Forbes_{year}.csv".format(year=i)
    f_data = pd.read_csv(f)
    
    com_ls = list(lda_now.company)
    
    forb_ls = get_forb(f_data,com_ls)
    
    lda_now["forb"] = forb_ls
    
    print(i)
    # 當年度正確預測比例
    # year_correct = len(lda_all[lda_all.iloc[:,2]==lda_all.iloc[:,3]])/len(lda_all)
    year_correct = len(lda_now[lda_now.iloc[:,1]==lda_now.iloc[:,2]])/len(lda_now)
    correct_ls.append(year_correct)
    
    # 當年度正確預測比例 去除4
    lda_now2 = lda_now[lda_now.iloc[:,1]==lda_now.iloc[:,2]]
    year_correct2 = len(lda_now2[lda_now2.iloc[:,2]!=4])/len(lda_now)
    correct_ls2.append(year_correct2)
    

2015
2016
2017
2018
2019
2020


In [13]:
correct_ls

[0.21428571428571427,
 0.07142857142857142,
 0.2857142857142857,
 0.21428571428571427,
 0.2857142857142857,
 0.5]

In [14]:
correct_ls2

[0.0,
 0.0,
 0.14285714285714285,
 0.14285714285714285,
 0.14285714285714285,
 0.35714285714285715]

In [15]:
change_ls = []

for i in range(2016,2021):
    now = "competitor_df/lda_cn_{year}.csv".format(year=i)
    befor = "competitor_df/lda_cn_{year}.csv".format(year=i-1)
    lda_now = pd.read_csv(now)
    lda_befor = pd.read_csv(befor)
    f = "../patent/code/依年分/Forbes/Forbes_{year}.csv".format(year=i)
    f_data = pd.read_csv(f)
    
    lda_all = lda_befor.iloc[:,0:2].merge(lda_now.iloc[:,0:2],on = 'company',how = 'outer')
    com_ls = list(lda_all.company)
    
    forb_ls = get_forb(f_data,com_ls)
    
    lda_all["forb"] = forb_ls 
    
    print(i)
    # 有變動且正確預測比例
    lda_all2 = lda_all[lda_all.iloc[:,2]==lda_all.iloc[:,3]] #先找出和forb相同的
    chan_correct = len(lda_all2[lda_all2.iloc[:,1]!=lda_all2.iloc[:,2]])/len(lda_all) #找出今年和前一年有變動的
    change_ls.append(chan_correct)
    
    

2016
2017
2018
2019
2020


In [16]:
change_ls

[0.05, 0.13636363636363635, 0.1, 0.15, 0.2]

In [38]:
lda_all2 = lda_all[lda_all.iloc[:,2]==lda_all.iloc[:,3]]
lda_all2

,company,2018_competitor,2019_competitor,forb
0,Qualcomm,1.0,1.0,1
1,Ericsson,1.0,2.0,2
2,Intel,1.0,1.0,1
4,Cisco Systems,1.0,2.0,2


In [42]:
lda_all2[(lda_all2.iloc[:,1]!=lda_all2.iloc[:,2])]

,company,2018_competitor,2019_competitor,forb
1,Ericsson,1.0,2.0,2
4,Cisco Systems,1.0,2.0,2


In [41]:
lda_all2[(lda_all2.iloc[:,1]!=lda_all2.iloc[:,2])|(lda_all2.iloc[:,1]==lda_all2.iloc[:,2])]

,company,2018_competitor,2019_competitor,forb
0,Qualcomm,1.0,1.0,1
1,Ericsson,1.0,2.0,2
2,Intel,1.0,1.0,1
4,Cisco Systems,1.0,2.0,2


In [40]:
lda_all2[lda_all2.iloc[:,1]==lda_all2.iloc[:,2]]

,company,2018_competitor,2019_competitor,forb
0,Qualcomm,1.0,1.0,1
2,Intel,1.0,1.0,1


In [27]:
lda_all[lda_all.iloc[:,1]!=lda_all.iloc[:,2]]

,company,2018_competitor,2019_competitor,forb
1,Ericsson,1.0,2.0,2
4,Cisco Systems,1.0,2.0,2
6,Apple,2.0,4.0,2
7,Nokia,2.0,4.0,2
8,LG Electronics,4.0,1.0,3
11,NEC,4.0,NaN,2
12,Amazon Technologies,4.0,NaN,4
14,Micron Technology,NaN,3.0,1
15,Huawei Technologies,NaN,2.0,4


In [9]:
lda_us = "../competitor_df/lda_us_{year}.csv".format(year="2019") 
lda_early = pd.read_csv(lda_us)

In [10]:
lda_early

,company,2019_competitor,2019_forb
0,Qualcomm,1,1
1,LG Electronics,1,3
2,IBM,1,2
3,Ericsson,2,2
4,Intel,1,1
5,Cisco Systems,2,2
6,Microsoft,1,2
7,Apple,4,2
8,AT&T,4,3
9,Micron Technology,3,1
